<a href="https://colab.research.google.com/github/rmajumder/w251-face-detection/blob/master/ImageRecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-jtlr7_8e
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-jtlr7_8e
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=788dde9bb2a5c8baa827e372b3c0b572abc1e0108c490c38c5f04c1ed8613a79
  Stored in directory: /tmp/pip-ephem-wheel-cache-z_nhykih/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


In [2]:
!pip install mtcnn

     |████████████████████████████████| 2.3MB 3.9MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257692 sha256=c636172f199cf48b25b6932e4ee215b7c716990b0843b259fc65af161d20b29e
  Stored in directory: /root/.cache/pip/wheels/85/81/65/6363fa5aafd7a155c896591e0c7c6e27b69642aa82b9cbf076
Successfully built mtcnn


In [3]:

# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
	# load image from file
	pixels = pyplot.imread(filename)
	# create the detector, using default weights
	detector = MTCNN()
	
	# detect faces in the image
	#pixels = Image.fromarray(pixels).resize((500,500))
	results = detector.detect_faces(pixels)
	
	# extract the bounding box from the first face
	x1, y1, width, height = results[0]['box']
	x2, y2 = x1 + width, y1 + height
	
	# extract the face
	face = pixels[y1:y2, x1:x2]
	# resize pixels to the model size
	image = Image.fromarray(face)
	image = image.resize(required_size)
	face_array = asarray(image)
	return face_array


# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
	# extract faces
	faces = [extract_face(f) for f in filenames]
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# create a vggface model
	model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
	# perform prediction
	yhat = model.predict(samples)
	return yhat

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))


In [6]:
!ls
!ls drive/"My Drive"/w251/w251

drive  sample_data
01.png		    face_recognition_tx2.py  rishi-240.png
Augmentation.ipynb  IdentifyFaceTX2.ipynb    rishi-60.png
ben1.png	    ImageRecognition.ipynb   rishi.jpeg
ben.png		    images-rishi	     rishi.png
CameraRoll	    README.md		     send_email_attach_aws_ses.py
cat1.png	    retrain_model.ipynb      sharon_stone1.jpg
channing_tatum.jpg  rishi-120.png	     sharon_stone2.jpg
dog1.png	    rishi-240.jpg	     sharon_stone3.jpg


In [7]:
# define filenames
filenames = ['drive/My Drive/w251/w251/rishi.jpeg', 'drive/My Drive/w251/w251/rishi-240.jpg',
	'drive/My Drive/w251/w251/sharon_stone3.jpg', 'drive/My Drive/w251/w251/channing_tatum.jpg']
# get embeddings file filenames
embeddings = get_embeddings(filenames)
# define sharon stone
sharon_id = embeddings[0]
# verify known photos of sharon
print('Positive Tests')
is_match(embeddings[0], embeddings[1])
is_match(embeddings[0], embeddings[2])
# verify known photos of other people
print('Negative Tests')
is_match(embeddings[0], embeddings[3])

W0807 05:05:37.919855 139936176129920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/mtcnn.py:187: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0807 05:05:37.959565 139936176129920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/mtcnn.py:193: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0807 05:05:38.029297 139936176129920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/network.py:43: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0807 05:05:38.031307 139936176129920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/layer_factory.py:88: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0807 05:05:38.040428 139936176129920 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/mtcnn/layer_factory.py:7

94699520/94694792 [==============================] - 3s 0us/step
Positive Tests
>face is a Match (0.065 <= 0.500)
>face is NOT a Match (0.833 > 0.500)
Negative Tests
>face is NOT a Match (0.686 > 0.500)


In [59]:
#title Adding protection against pictures of people
def is_match_is_picture(known_embedding, candidate_embedding, picture_of_candidate, thresh=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	if score <= thresh:
		picture_score= cosine(known_embedding, picture_of_candidate)
		if score < picture_score:
			print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
			return 1
		else:
			print('>face is a picture (%.3f <= %.3f)'% (score, picture_score))
			return 0
	else:
		print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))
		return -100

# define filenames
filenames = ['drive/My Drive/w251/w251/CameraRoll/rodrigo_webcam.jpg',
'drive/My Drive/w251/w251/CameraRoll/rigo_webcam_down.jpg',
'drive/My Drive/w251/w251/CameraRoll/rigo_webcam_side.jpg',	
'drive/My Drive/w251/w251/CameraRoll/IMG-20190806-WA0022.jpg',
'drive/My Drive/w251/w251/CameraRoll/IMG-20190806-WA0020.jpg',
'drive/My Drive/w251/w251/CameraRoll/IMG-20190806-WA0021.jpg',
'drive/My Drive/w251/w251/CameraRoll/rodrigo_pic_of_pic.jpg',
'drive/My Drive/w251/w251/CameraRoll/rigo_pic_of_pic.jpg','drive/My Drive/w251/w251/CameraRoll/pic_of_pic_1.jpg','drive/My Drive/w251/w251/CameraRoll/pic_of_pic3.jpg',
'drive/My Drive/w251/w251/CameraRoll/pic_of_pic_d2.jpg']

# get embeddings file filenames
embeddings_pics = get_embeddings(filenames)
# define sharon stone
sharon_id = embeddings[0]
# verify known photos of sharon
print('Positive Tests')


Positive Tests


In [64]:
webcams=[0,1,2]
phones=[3,4,5]
webcam2phone=[6,7,8,9,10]
p1=0;p2=0;p3=0
#webcam, webcam, webcam2phone
is_match_is_picture(embeddings_pics[0], embeddings_pics[2], embeddings_pics[8], thresh=0.5)
#phone, phone, webcam2phone
is_match_is_picture(embeddings_pics[3], embeddings_pics[4], embeddings_pics[8], thresh=0.5)
#phone, webcam, webcam2phone
count1=0
for w2p in webcam2phone:
  for webcam in webcams:
    for phone in phones:
      count1=count1+1
      p1=p1+is_match_is_picture(embeddings_pics[phone], embeddings_pics[webcam], embeddings_pics[w2p], thresh=0.5)

# verify known photos of other people
print('Negative Tests')
count2=0

for w2p in webcam2phone:
  for w2p2 in [w2p2  for w2p2 in webcam2phone if w2p is not w2p2]:
    for webcam in webcams:
#web, webcam2phone, webcam2phone
      count2=count2+1 
      p2=p2+is_match_is_picture(embeddings_pics[webcam], embeddings_pics[w2p], embeddings_pics[w2p2], thresh=0.5)
#phone, webcam2phone, webcam2phone

for w2p in webcam2phone:
  for w2p2 in [w2p2 for w2p2 in webcam2phone if w2p is not w2p2]:
    for phone in phones:
      p3=p3+is_match_is_picture(embeddings_pics[phone], embeddings_pics[w2p], embeddings_pics[w2p2], thresh=0.5)
print(p1/count1, p2/count2, p3/count2)

>face is a Match (0.173 <= 0.500)
>face is a picture (0.291 <= 0.150)
>face is a picture (0.176 <= 0.076)
>face is a Match (0.240 <= 0.500)
>face is a Match (0.337 <= 0.500)
>face is a picture (0.226 <= 0.076)
>face is a Match (0.238 <= 0.500)
>face is a picture (0.354 <= 0.351)
>face is a picture (0.225 <= 0.076)
>face is a picture (0.362 <= 0.347)
>face is a picture (0.363 <= 0.351)
>face is a picture (0.176 <= 0.143)
>face is a Match (0.240 <= 0.500)
>face is a Match (0.337 <= 0.500)
>face is a picture (0.226 <= 0.143)
>face is a Match (0.238 <= 0.500)
>face is a Match (0.354 <= 0.500)
>face is a picture (0.225 <= 0.143)
>face is a picture (0.362 <= 0.329)
>face is a picture (0.363 <= 0.358)
>face is a picture (0.176 <= 0.150)
>face is a Match (0.240 <= 0.500)
>face is a Match (0.337 <= 0.500)
>face is a picture (0.226 <= 0.150)
>face is a Match (0.238 <= 0.500)
>face is a picture (0.354 <= 0.352)
>face is a picture (0.225 <= 0.150)
>face is a Match (0.362 <= 0.500)
>face is a pictu

In [39]:
!ls drive/"My Drive"/w251/w251/CameraRoll/ #
'''
drive/My Drive/w251/w251/CameraRoll/rodrigo_webcam.jpg
drive/My Drive/w251/w251/CameraRoll/rigo_webcam_down.jpg
drive/My Drive/w251/w251/CameraRoll/rigo_webcam_side.jpg	
drive/My Drive/w251/w251/CameraRoll/rodrigo_pic_of_pic.jpg
drive/My Drive/w251/w251/CameraRoll/rigo_pic_of_pic.jpg
drive/My Drive/w251/w251/CameraRoll/rigo_orig_pic.jpg
drive/My Drive/w251/w251/CameraRoll/rigo_orig_pic_II.jpg
'''

desktop.ini		 pic_of_pic2.jpg       rodrigo_pic_of_pic.jpg
IMG-20190806-WA0019.jpg  pic_of_pic3.jpg       rodrigo_webcam.jpg
IMG-20190806-WA0020.jpg  pic_of_pic_d1.jpg     WIN_20190806_21_57_36_Pro.jpg
IMG-20190806-WA0021.jpg  pic_of_pic_d2.jpg     WIN_20190806_21_57_45_Pro.jpg
IMG-20190806-WA0022.jpg  rigo_orig_pic_II.jpg  WIN_20190806_21_57_51_Pro.jpg
IMG-20190806-WA0023.jpg  rigo_orig_pic.jpg     WIN_20190806_21_58_02_Pro.jpg
IMG-20190806-WA0024.jpg  rigo_pic_of_pic.jpg   WIN_20190806_21_58_34_Pro.jpg
IMG-20190806-WA0025.jpg  rigo_webcam_down.jpg
pic_of_pic_1.jpg	 rigo_webcam_side.jpg


'\ndrive/My Drive/w251/w251/CameraRoll/rodrigo_webcam.jpg\ndrive/My Drive/w251/w251/CameraRoll/rigo_webcam_down.jpg\ndrive/My Drive/w251/w251/CameraRoll/rigo_webcam_side.jpg\t\ndrive/My Drive/w251/w251/CameraRoll/rodrigo_pic_of_pic.jpg\ndrive/My Drive/w251/w251/CameraRoll/rigo_pic_of_pic.jpg\ndrive/My Drive/w251/w251/CameraRoll/rigo_orig_pic.jpg\ndrive/My Drive/w251/w251/CameraRoll/rigo_orig_pic_II.jpg\n'